In [ ]:
%load_ext nb_black

In [ ]:
from node2vec_def.randomwalk import GraphEmbedding

In [ ]:
import os
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import torch
from torch import Tensor
import random
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
import networkx as nx
from networkx.algorithms import community
import numpy as np
from gensim.models import Word2Vec
import networkx as nx
import random

In [ ]:
def loss_function(graph, params):
    # Create an instance of GraphEmbedding with the provided parameters
    graph_embedding = GraphEmbedding(graph, params)

    # Compute transition probabilities and generate random walks
    graph_embedding.transition_probabilities()
    walks, S = graph_embedding.random_walks()

    # Train node embeddings using Word2Vec
    embeddings = graph_embedding.train_embedding(walks, params)

    # Calculate the objective function
    Z = embeddings.vectors.T
    Z_transpose = np.transpose(Z)
    diff = np.dot(Z_transpose, Z) - S
    loss_func = np.linalg.norm(diff)

    return loss_func

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "Data/"
os.makedirs(data_dir, exist_ok=True)
SEED = 42

In [ ]:
dataset = Planetoid(root=data_dir, name="Cora")
data = dataset[0]

In [4]:
import numpy as np
np.random.choice([0.01, 0.1, 0.001,0.0001], 10)

array([0.1   , 0.0001, 0.0001, 0.1   , 0.1   , 0.01  , 0.001 , 0.0001,
       0.1   , 0.001 ])

In [13]:
np.random.uniform(0.0, 4.0, 50)

array([0.3954995 , 1.08851884, 3.29159999, 1.64117367, 3.84471808,
       0.55133203, 1.66854335, 0.70950104, 2.32040086, 2.0576166 ,
       3.6590112 , 3.51967905, 0.90412677, 2.55373185, 2.94813562,
       0.9351806 , 1.37018046, 0.42081767, 0.47220347, 3.98646609,
       0.55773281, 2.30218931, 1.81357128, 1.74829798, 0.25368243,
       1.68290284, 2.82935655, 0.16473638, 2.88389399, 3.30561912,
       1.30199194, 0.77321901, 2.32314872, 2.81440336, 0.69992628,
       3.887801  , 0.80852634, 0.47160114, 1.18222454, 1.32204467,
       3.50861822, 0.01490749, 1.79635725, 1.55414206, 0.7275717 ,
       1.77557039, 1.86872234, 1.75445646, 1.81705467, 1.69512143])

In [14]:
import datetime

In [25]:
today_date = datetime.date.today().strftime("%d_%m_%Y")

In [26]:
today_date

'08_09_2023'

In [ ]:
    def generate_random_walks(self):
        G = self.graph
        random_walks = []
        num_nodes = len(G.nodes())
        similarity_matrix = torch.zeros((num_nodes, num_nodes), device='cuda' if torch.cuda.is_available() else 'cpu')

        for start_node in tqdm(G.nodes(), desc="Generating Random Walks"):
            for i in range(self.num_walks):
                walk = [start_node]
                walk_options = list(G[start_node])

                if len(walk_options) == 0:
                    break
                first_step = np.random.choice(walk_options)
                walk.append(first_step)

                for _ in range(self.walk_length - 2):
                    walk_options = list(G[walk[-1]])
                    if len(walk_options) == 0:
                        break
                    current_node = walk[-1]
                    probabilities = self.transition_probs[walk[-2]]["probabilities"][current_node]

                    # Move probabilities to the GPU if available
                    probabilities = torch.tensor(probabilities, device='cuda' if torch.cuda.is_available() else 'cpu')

                    next_step = np.random.choice(walk_options, p=probabilities.cpu().numpy())
                    walk.append(next_step)

                    # Update the similarity matrix
                    visited_nodes = set(walk)
                    for node_j in G.nodes():
                        if node_j == start_node:
                            similarity_matrix[start_node][node_j] = 1.00
                        elif node_j not in visited_nodes:
                            similarity_matrix[start_node][node_j] = 0.00
                        else:
                            prob_ij = walk.count(node_j) / (self.walk_length - 1)
                            similarity_matrix[start_node][node_j] += prob_ij

                random_walks.append(walk)

        np.random.shuffle(random_walks)

        random_walks = [list(map(str, walk)) for walk in random_walks]

        return random_walks, similarity_matrix

In [ ]:
np.random.choice([32, 64, 6], 5)

In [ ]:
data

In [ ]:
G = to_networkx(data, to_undirected=True)

In [ ]:
[
    [1,2,3],
    [1,2,4],
    [2,1,3],
    [2,4,2],
    [3,1,4],
    [3,1,2],
    [4,1,4],
    [4,2,1]
]